In [2]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, GlobalAveragePooling2D, Input
from tensorflow.keras.preprocessing.image import img_to_array

In [3]:
#Load train and test csv files
train_df = pd.read_csv('../CSV Files/train.csv')
test_df = pd.read_csv('../CSV Files/test.csv')

### Extract frames from videos

In [4]:
def extract_frames(video_path, num_frames=30):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_interval = max(1, total_frames // num_frames)
    
    for i in range(num_frames):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * frame_interval)
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (299, 299))
        frame = img_to_array(frame) / 255.0
        frames.append(frame)
    
    cap.release()
    return np.array(frames)

### Data set preparation

In [5]:
def prepare_dataset(df, num_frames=30):
    X, y = [], []
    for index, row in df.iterrows():
        video_path, label = row['Video Name'], row['Tag']
        frames = extract_frames(video_path, num_frames)
        if frames.shape[0] == num_frames:
            X.append(frames)
            y.append(1 if label.lower() == "fake" else 0)  
    return np.array(X), np.array(y, dtype=np.float32) 

In [6]:
# Prepare training data
X_train, y_train = prepare_dataset(train_df)

In [7]:
#Prepare testing data
X_test,y_test = prepare_dataset(test_df)

## Defining Model Architecture

In [8]:
input_layer = Input(shape=(30, 299, 299, 3))  # 30 frames of size 299x299x3

In [9]:
# Xception base model for feature extraction
base_model = Xception(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
base_model.trainable = False

In [10]:
# TimeDistributed wrapper to apply Xception on each frame
x = TimeDistributed(base_model)(input_layer)
x = TimeDistributed(GlobalAveragePooling2D())(x)

In [11]:
# LSTM layers for sequential analysis
x = LSTM(128, return_sequences=True)(x)
x = LSTM(64, return_sequences=False)(x)

In [12]:
# Fully connected layers
x = Dense(64, activation='relu')(x)
out = Dense(1, activation='sigmoid')(x)

In [13]:
# Build and compile the model
model = Model(inputs=input_layer, outputs=out)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
# Train model
model.fit(X_train, y_train, epochs=5, batch_size=8, validation_split=0.2)

Epoch 1/5
34/34 ━━━━━━━━━━━━━━━━━━━━ 997s 29s/step - accuracy: 0.6303 - loss: 0.6638 - val_accuracy: 0.5735 - val_loss: 0.6932
Epoch 2/5
34/34 ━━━━━━━━━━━━━━━━━━━━ 932s 27s/step - accuracy: 0.5937 - loss: 0.6544 - val_accuracy: 0.4118 - val_loss: 0.9085
Epoch 3/5
34/34 ━━━━━━━━━━━━━━━━━━━━ 851s 25s/step - accuracy: 0.6050 - loss: 0.6515 - val_accuracy: 0.4118 - val_loss: 0.9192
Epoch 4/5
34/34 ━━━━━━━━━━━━━━━━━━━━ 811s 24s/step - accuracy: 0.6727 - loss: 0.6049 - val_accuracy: 0.0441 - val_loss: 0.8989
Epoch 5/5
34/34 ━━━━━━━━━━━━━━━━━━━━ 824s 24s/step - accuracy: 0.7071 - loss: 0.5577 - val_accuracy: 0.6618 - val_loss: 0.6783


In [15]:
# Evaluate model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 148s 66s/step - accuracy: 0.6481 - loss: 0.5758
Test Loss: 0.5956, Test Accuracy: 0.6441


In [17]:
# Save the model
model.save("../Models/xception_lstm.keras")
print("Model saved as xception_lstm.keras")

Model saved as xception_lstm.keras


In [18]:
model.save("../Models/Xception_LSTM/xception_lstm.keras", save_format="keras")
